# Task 4.5.3: Prototype Patterns - Building Demos That Impress

**Module:** 4.5 - Demo Building & Prototyping  
**Time:** 1-2 hours  
**Difficulty:** ⭐⭐☆☆☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the difference between demos and production systems
- [ ] Master patterns for rapid prototype development
- [ ] Know when and how to cut corners effectively
- [ ] Communicate effectively with technical and non-technical stakeholders
- [ ] Handle errors gracefully in demo contexts

---

## 📚 Prerequisites

- Completed: Tasks 4.5.1-4.5.2 (Gradio and Streamlit basics)
- Experience: Built at least one demo or prototype

---

## 🌍 Real-World Context

You've been working on a RAG system for 3 months. Your VP schedules a demo for the board next week. The system works... mostly. What do you show? What do you hide? How do you present 3 months of complex work in 15 minutes?

This is the art of demo building - and it's a skill that can make or break your projects.

---

## 🧒 ELI5: Demo vs Production

> **A demo** is like a movie trailer. It shows the best parts, hides the boring bits, and gets people excited. Nobody expects the full movie in 2 minutes.
>
> **Production** is the full movie. Every scene needs to work. No shortcuts. No "we'll fix it in post."
>
> The mistake people make: treating a trailer like a movie (too much detail) or a movie like a trailer (not enough reliability).
>
> **The golden rule:** A demo should be just good enough to get people excited and *just reliable enough* not to crash during the presentation.

---

## Part 1: Demo ≠ Production

### What to Polish vs What to Skip

| **POLISH** (High Impact) | **SKIP** (Low Impact for Demo) |
|--------------------------|--------------------------------|
| First impression (landing page) | Database optimization |
| Happy path works flawlessly | Edge case handling |
| Error messages are friendly | Comprehensive logging |
| Loading states look good | Background job processing |
| Example inputs pre-loaded | User authentication |
| Results look impressive | Rate limiting |

### The Demo Checklist

Before any demo, ask:
1. ✅ Does the happy path work 100% of the time?
2. ✅ Are there pre-loaded examples that always work?
3. ✅ Do errors show friendly messages (not stack traces)?
4. ✅ Does it look professional (not like a developer tool)?
5. ✅ Can a non-technical person use it without guidance?

In [ ]:
# Example: Demo vs Production Error Handling

# ❌ PRODUCTION CODE - Technical, detailed errors
def production_query(question):
    try:
        result = rag_system.query(question)
        return result
    except ConnectionError as e:
        raise HTTPException(status_code=503, detail=f"Database connection failed: {str(e)}")
    except ValidationError as e:
        raise HTTPException(status_code=400, detail=f"Invalid query format: {e.json()}")
    except Exception as e:
        logger.exception("Unexpected error in query")
        raise HTTPException(status_code=500, detail="Internal server error")

# ✅ DEMO CODE - Friendly, simple errors
def demo_query(question):
    try:
        result = rag_system.query(question)
        return result
    except Exception as e:
        # Never show the real error in a demo!
        return "I'm having trouble processing that question. Could you try rephrasing it? 🤔"

print("Demo code prioritizes user experience over technical accuracy.")
print("Production code prioritizes correctness and debugging over UX.")

---

## Part 2: The "Golden Path" Pattern

### 🧒 ELI5: Golden Path

> Imagine you're giving a tour of a fancy hotel. You show the beautiful lobby, the best suite, the rooftop pool. You DON'T show the laundry room, the boiler, or the room being renovated.
>
> The "golden path" is the tour route you design - the sequence of features that always works and always impresses.

### Pre-Loaded Examples

Always have examples that:
1. **Work 100% of the time** (tested extensively)
2. **Show off the best features** (not edge cases)
3. **Are relevant to the audience** (executives care about business value, engineers care about tech)
4. **Complete quickly** (nobody wants to wait 30 seconds in a demo)

In [ ]:
import gradio as gr

# Golden path pattern: Pre-loaded examples with guaranteed results

# Define examples that ALWAYS work
GOLDEN_EXAMPLES = [
    {
        "question": "What are the key features of our new product?",
        "expected_response": "Our new product features...",  # Pre-verified
        "audience": "executives"
    },
    {
        "question": "How does the caching layer improve performance?",
        "expected_response": "The caching layer reduces...",
        "audience": "engineers"
    },
    {
        "question": "Summarize our Q3 earnings report",
        "expected_response": "Q3 earnings showed...",
        "audience": "investors"
    }
]

def create_demo_with_examples():
    """Create a Gradio demo with golden path examples."""
    
    def respond(question):
        # In a real demo, this calls your RAG system
        # For golden examples, you might even cache the exact response!
        return f"Response to: {question}"
    
    demo = gr.Interface(
        fn=respond,
        inputs=gr.Textbox(label="Ask a Question", placeholder="Try one of the examples below!"),
        outputs=gr.Textbox(label="Response"),
        title="📚 Document Q&A Demo",
        description="Ask questions about your documents.",
        # These appear as clickable buttons!
        examples=[
            [ex["question"]] for ex in GOLDEN_EXAMPLES
        ],
        cache_examples=True  # Pre-compute responses for examples!
    )
    
    return demo

print("Golden examples ensure your demo always has something that works!")

### The "Demo Mode" Toggle

A useful pattern: have a "demo mode" that uses cached/safe responses.

In [ ]:
# Demo mode pattern

DEMO_RESPONSES = {
    "What is our company's mission?": 
        "Our mission is to democratize AI for everyone, making powerful AI tools accessible and safe.",
    "How do I reset my password?": 
        "To reset your password, click 'Forgot Password' on the login page and follow the email instructions.",
    "What's new in version 2.0?": 
        "Version 2.0 introduces: 1) 3x faster response times, 2) Multi-language support, 3) Custom integrations."
}

class DemoAwareRAG:
    """RAG system with demo mode."""
    
    def __init__(self, demo_mode: bool = False):
        self.demo_mode = demo_mode
    
    def query(self, question: str) -> str:
        # In demo mode, use pre-cached responses for known questions
        if self.demo_mode and question in DEMO_RESPONSES:
            return DEMO_RESPONSES[question]
        
        # Otherwise, run the real system
        return self._real_query(question)
    
    def _real_query(self, question: str) -> str:
        # Your actual RAG implementation
        return f"Real response for: {question}"

# Usage
rag = DemoAwareRAG(demo_mode=True)
print(rag.query("What is our company's mission?"))

---

## Part 3: Progressive Disclosure

### 🧒 ELI5: Progressive Disclosure

> Think of a good video game tutorial. It doesn't dump all controls on you at once. First: "Press A to jump." Later: "Hold B to run." Even later: "Press X+Y for special move."
>
> Your demo should work the same way. Start simple, reveal complexity only when needed.

### Implementation Pattern

In [ ]:
import gradio as gr

def create_progressive_demo():
    """Demo with progressive disclosure - simple first, advanced later."""
    
    with gr.Blocks() as demo:
        gr.Markdown("# 🤖 AI Assistant")
        
        # LEVEL 1: The basics (always visible)
        with gr.Group():
            query = gr.Textbox(label="Your Question", placeholder="Ask anything...")
            submit = gr.Button("Ask", variant="primary")
            response = gr.Textbox(label="Response", lines=5)
        
        # LEVEL 2: Slightly advanced (collapsed by default)
        with gr.Accordion("⚙️ Options", open=False):
            model = gr.Dropdown(
                choices=["Fast (8B)", "Balanced (13B)", "Best (70B)"],
                value="Balanced (13B)",
                label="Model Quality"
            )
            creativity = gr.Slider(0, 1, 0.7, label="Creativity")
        
        # LEVEL 3: Power user features (very hidden)
        with gr.Accordion("🔧 Advanced Settings", open=False):
            gr.Markdown("*For power users only*")
            system_prompt = gr.Textbox(
                label="System Prompt",
                value="You are a helpful assistant.",
                lines=3
            )
            max_tokens = gr.Number(label="Max Tokens", value=500)
            show_sources = gr.Checkbox(label="Show Sources", value=False)
        
        # LEVEL 4: Debug info (only for developers)
        with gr.Accordion("🐛 Debug Info", open=False, visible=True):
            debug_output = gr.JSON(label="Raw Response")
        
        def process(query, model, creativity, system_prompt, max_tokens, show_sources):
            # Your processing logic
            result = f"Response to: {query} (model: {model}, creativity: {creativity})"
            debug = {
                "query": query,
                "model": model,
                "tokens_used": 42,
                "latency_ms": 150
            }
            return result, debug
        
        submit.click(
            process,
            inputs=[query, model, creativity, system_prompt, max_tokens, show_sources],
            outputs=[response, debug_output]
        )
    
    return demo

print("Progressive disclosure: Simple for beginners, powerful for experts.")

---

## Part 4: Graceful Error Handling

In demos, errors WILL happen. The question is: how embarrassing will they be?

### Error Handling Strategies

In [ ]:
# Error handling patterns for demos

import random

# Strategy 1: Friendly error messages
FRIENDLY_ERRORS = [
    "Hmm, I'm having trouble with that. Could you try asking differently? 🤔",
    "That's a tricky one! Let me suggest trying a simpler question first.",
    "I'm still learning about that topic. Try one of the example questions!",
    "Oops! Something went sideways. Let's try again with a different question."
]

def safe_query(query_fn):
    """Decorator that catches errors and returns friendly messages."""
    def wrapper(question, *args, **kwargs):
        try:
            return query_fn(question, *args, **kwargs)
        except Exception as e:
            # Log the real error (but don't show it)
            print(f"ERROR (hidden from user): {e}")
            # Return a friendly message
            return random.choice(FRIENDLY_ERRORS)
    return wrapper

# Strategy 2: Fallback responses
def query_with_fallback(question):
    """Try multiple strategies, fallback gracefully."""
    
    # Try the main method
    try:
        return main_rag_query(question)
    except Exception:
        pass
    
    # Fallback 1: Simpler model
    try:
        return simple_model_query(question)
    except Exception:
        pass
    
    # Fallback 2: Keyword search
    try:
        return keyword_search(question)
    except Exception:
        pass
    
    # Final fallback: Apologetic message
    return "I apologize, but I'm unable to answer that right now. Please try one of our example questions!"

# Mock functions for demonstration
def main_rag_query(q): raise Exception("Main system down")
def simple_model_query(q): return f"Simple answer for: {q}"
def keyword_search(q): return f"Keyword results for: {q}"

# Test
print(query_with_fallback("What is AI?"))

In [ ]:
# Strategy 3: Timeout handling with loading states

import gradio as gr
import time

def create_timeout_aware_demo():
    """Demo with proper timeout handling."""
    
    def slow_query(question):
        """Simulates a potentially slow query."""
        import threading
        result = [None]
        error = [None]
        
        def run():
            try:
                time.sleep(2)  # Simulate processing
                result[0] = f"Answer to: {question}"
            except Exception as e:
                error[0] = e
        
        thread = threading.Thread(target=run)
        thread.start()
        thread.join(timeout=5)  # 5 second timeout
        
        if thread.is_alive():
            return "⏱️ That's taking longer than expected. The system might be busy. Try again?"
        
        if error[0]:
            return "Something went wrong. Please try a different question."
        
        return result[0]
    
    demo = gr.Interface(
        fn=slow_query,
        inputs=gr.Textbox(label="Question"),
        outputs=gr.Textbox(label="Answer"),
        title="Timeout-Aware Demo"
    )
    
    return demo

print("Always handle timeouts - demos shouldn't hang forever!")

---

## Part 5: Stakeholder Communication

### Know Your Audience

| Audience | They Care About | Show Them | Avoid |
|----------|-----------------|-----------|-------|
| **Executives** | Business value, ROI | Use cases, metrics, time saved | Technical details |
| **Engineers** | Architecture, scalability | System design, code quality | Business jargon |
| **Users** | Ease of use, reliability | The interface, examples | Backend complexity |
| **Investors** | Market potential, growth | Traction, vision | Current limitations |

### The Demo Script

Always have a script (even if you don't follow it exactly):

1. **Hook** (30 sec): "Imagine you could search 10 years of documents in seconds..."
2. **Problem** (1 min): "Currently, this takes 3 hours and misses 40% of relevant info."
3. **Solution** (3-5 min): Live demo of the golden path
4. **Wow moment** (1 min): Show something impressive they didn't expect
5. **Call to action** (30 sec): "We need X to take this to production."

In [ ]:
# Audience-specific demo configuration

AUDIENCE_CONFIGS = {
    "executive": {
        "show_technical_details": False,
        "show_metrics": True,
        "show_business_value": True,
        "example_questions": [
            "What were our top selling products last quarter?",
            "Summarize the competitive landscape report.",
            "What are the key risks in our expansion plan?"
        ],
        "metrics_to_highlight": ["time_saved", "accuracy", "cost_reduction"]
    },
    "engineer": {
        "show_technical_details": True,
        "show_metrics": True,
        "show_business_value": False,
        "example_questions": [
            "How does the embedding model handle domain-specific terms?",
            "What's the latency breakdown for a typical query?",
            "Show me the retrieval pipeline architecture."
        ],
        "metrics_to_highlight": ["latency_p99", "retrieval_precision", "tokens_per_query"]
    },
    "user": {
        "show_technical_details": False,
        "show_metrics": False,
        "show_business_value": False,
        "example_questions": [
            "How do I submit an expense report?",
            "What's the vacation policy?",
            "Find the latest product roadmap."
        ],
        "metrics_to_highlight": []
    }
}

def configure_demo_for_audience(audience: str):
    """Return demo configuration for the target audience."""
    config = AUDIENCE_CONFIGS.get(audience, AUDIENCE_CONFIGS["user"])
    print(f"\nDemo configured for: {audience.upper()}")
    print(f"  - Show technical details: {config['show_technical_details']}")
    print(f"  - Example questions: {config['example_questions'][0]}...")
    return config

# Example
exec_config = configure_demo_for_audience("executive")
eng_config = configure_demo_for_audience("engineer")

---

## Part 6: Rapid Iteration Patterns

### The MVP Mindset

**M**inimum **V**iable **P**roduct - what's the least you can build to test your idea?

### Iteration Cycle

```
1. Build (2-4 hours) → Quick prototype
2. Demo (30 min) → Show to users/stakeholders  
3. Feedback (30 min) → What worked? What didn't?
4. Prioritize (30 min) → What's most impactful to fix?
5. Repeat
```

### Feature Prioritization Matrix

| | **Easy to Build** | **Hard to Build** |
|---|-------------------|-------------------|
| **High Impact** | DO FIRST 🚀 | Schedule carefully 📅 |
| **Low Impact** | Do if time allows 🤷 | Skip for now ❌ |

In [ ]:
# Feature prioritization helper

FEATURES = [
    {"name": "Chat interface", "impact": 9, "effort": 3},
    {"name": "File upload", "impact": 7, "effort": 2},
    {"name": "Source citations", "impact": 6, "effort": 4},
    {"name": "User authentication", "impact": 3, "effort": 7},
    {"name": "Analytics dashboard", "impact": 4, "effort": 8},
    {"name": "Dark mode", "impact": 2, "effort": 1},
    {"name": "Export to PDF", "impact": 3, "effort": 3},
    {"name": "Multi-language", "impact": 5, "effort": 9},
]

def prioritize_features(features, max_effort=10):
    """
    Prioritize features by impact/effort ratio.
    Returns features sorted by priority.
    """
    # Calculate priority score (impact / effort)
    for f in features:
        f["priority_score"] = f["impact"] / f["effort"]
        f["quadrant"] = get_quadrant(f["impact"], f["effort"])
    
    # Sort by priority score
    sorted_features = sorted(features, key=lambda x: x["priority_score"], reverse=True)
    
    return sorted_features

def get_quadrant(impact, effort):
    """Determine which quadrant a feature falls into."""
    if impact >= 5 and effort <= 5:
        return "🚀 DO FIRST"
    elif impact >= 5 and effort > 5:
        return "📅 SCHEDULE"
    elif impact < 5 and effort <= 5:
        return "🤷 MAYBE"
    else:
        return "❌ SKIP"

# Prioritize
prioritized = prioritize_features(FEATURES)

print("Feature Priority for Demo MVP:")
print("=" * 60)
for i, f in enumerate(prioritized, 1):
    print(f"{i}. {f['name']:25} | Score: {f['priority_score']:.1f} | {f['quadrant']}")

---

## Part 7: Complete Demo Template

Here's a complete, production-ready demo template with all best practices:

In [ ]:
# Complete demo template with all best practices

demo_template = '''
import gradio as gr
import time
import random

# ============================================================================
# CONFIGURATION
# ============================================================================

DEMO_MODE = True  # Set to False for production

# Golden path examples (always work!)
GOLDEN_EXAMPLES = [
    "What are our company's core values?",
    "Summarize the Q3 earnings report.",
    "How do I request time off?"
]

# Pre-cached responses for demo mode
DEMO_RESPONSES = {
    GOLDEN_EXAMPLES[0]: "Our core values are: Innovation, Integrity, Customer Focus, and Teamwork.",
    GOLDEN_EXAMPLES[1]: "Q3 showed 15% revenue growth, with strong performance in cloud services.",
    GOLDEN_EXAMPLES[2]: "To request time off, go to HR Portal > Time Off > Submit Request."
}

# Friendly error messages
FRIENDLY_ERRORS = [
    "I\'m having trouble with that question. Could you try one of the examples?",
    "Let me suggest trying a different phrasing.",
    "That\'s outside my current knowledge. Try asking about company policies or reports!"
]

# ============================================================================
# CUSTOM THEME (Optional - for branding)
# ============================================================================

custom_theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="gray",
).set(
    button_primary_background_fill="#0066cc",
    button_primary_text_color="white",
)

# ============================================================================
# CORE LOGIC
# ============================================================================

def safe_query(question: str) -> str:
    """
    Safe wrapper for the RAG query with fallbacks.
    """
    # Check for demo mode cached responses
    if DEMO_MODE and question in DEMO_RESPONSES:
        time.sleep(0.5)  # Simulate processing
        return DEMO_RESPONSES[question]
    
    try:
        # Your actual RAG query here
        # result = rag_system.query(question)
        time.sleep(1)  # Simulate processing
        result = f"Response for: {question}"
        return result
    except Exception as e:
        print(f"Error (hidden): {e}")
        return random.choice(FRIENDLY_ERRORS)

# ============================================================================
# INTERFACE
# ============================================================================

with gr.Blocks(theme=custom_theme, title="Document Q&A") as demo:
    # Header
    gr.Markdown("""
    # 📚 Document Q&A Assistant
    
    Ask questions about company documents, policies, and reports.
    """)
    
    # Main interface (simple, prominent)
    with gr.Row():
        with gr.Column(scale=3):
            question = gr.Textbox(
                label="Your Question",
                placeholder="Ask anything about our documents...",
                lines=2
            )
            submit = gr.Button("Ask", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            gr.Markdown("### Quick Examples")
            for ex in GOLDEN_EXAMPLES:
                gr.Button(ex[:30] + "...", size="sm").click(
                    lambda e=ex: e, None, question
                )
    
    # Response area
    response = gr.Textbox(label="Answer", lines=6, interactive=False)
    
    # Advanced options (hidden by default)
    with gr.Accordion("⚙️ Advanced Options", open=False):
        with gr.Row():
            model = gr.Dropdown(
                choices=["Fast (8B)", "Balanced (13B)", "Best (70B)"],
                value="Balanced (13B)",
                label="Model"
            )
            temperature = gr.Slider(0, 1, 0.7, label="Creativity")
    
    # Footer
    gr.Markdown("---")
    gr.Markdown("_Powered by RAG + Llama 3.1 | Demo Version_", elem_classes="footer")
    
    # Event handlers
    submit.click(safe_query, [question], [response])
    question.submit(safe_query, [question], [response])

# ============================================================================
# LAUNCH
# ============================================================================

if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",  # Allow external access
        server_port=7860,
        share=False,  # Set True for public link
        show_error=False  # Hide errors in production
    )
'''

# Save the template
with open('/tmp/demo_template.py', 'w') as f:
    f.write(demo_template)

print("Complete demo template saved to /tmp/demo_template.py")
print("\nThis template includes:")
print("  ✅ Demo mode with cached responses")
print("  ✅ Golden path examples")
print("  ✅ Graceful error handling")
print("  ✅ Progressive disclosure")
print("  ✅ Custom theming")
print("  ✅ Production-ready launch config")

---

## ⚠️ Common Mistakes

### Mistake 1: Showing Technical Errors
```python
# ❌ Exposing stack traces
except Exception as e:
    return f"Error: {traceback.format_exc()}"

# ✅ Friendly messages
except Exception as e:
    logger.error(f"Hidden: {e}")
    return "I couldn't process that. Try a different question!"
```

### Mistake 2: No Backup Plan
```python
# ❌ Single point of failure
response = main_model.generate(prompt)  # What if this fails?

# ✅ Multiple fallbacks
response = try_main() or try_fallback() or SAFE_DEFAULT_RESPONSE
```

### Mistake 3: Demo-ing Unpolished Features
```python
# ❌ "Oh, that button doesn't work yet..."
# ❌ "Let me just restart the server..."
# ❌ "That feature is buggy, skip it..."

# ✅ Only show what works perfectly
# ✅ Remove unfinished features from demo
# ✅ Test the exact flow before presenting
```

### Mistake 4: Too Much Complexity
```python
# ❌ Showing everything
"Let me show you the 47 configuration options..."

# ✅ Progressive disclosure
"Here's the main feature. Advanced options are available if needed."
```

---

## 🎉 Checkpoint

You've learned:
- ✅ When to polish vs when to skip (demo ≠ production)
- ✅ The golden path pattern with pre-loaded examples
- ✅ Progressive disclosure for complexity management
- ✅ Graceful error handling strategies
- ✅ Audience-specific communication
- ✅ MVP mindset and feature prioritization

---

## 🚀 Challenge (Optional)

Create a **Demo Checklist Tool** that:
1. Takes a list of features as input
2. Asks about impact and effort for each
3. Generates a prioritized backlog
4. Creates a "golden path" script for the demo

---

## 📖 Further Reading

- [The Mom Test](http://momtestbook.com/) - How to get honest feedback
- [Demo or Die](https://www.youtube.com/watch?v=Hc16Cxn_gGk) - MIT Media Lab philosophy
- [Don't Make Me Think](https://sensible.com/dont-make-me-think/) - UX principles

---

## 🧹 Cleanup

In [ ]:
# List files created in this notebook
import os

files_created = [
    '/tmp/demo_template.py'
]

print("Files created:")
for f in files_created:
    if os.path.exists(f):
        print(f"  ✅ {f}")

print("\n✅ Ready for the next task!")